In [279]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [280]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]

data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [281]:
# Copy the dataframe
df2 = data.copy()

In [282]:
#Scale for better intepretation

df2['Price'] = df2['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
df2['HP'] = df2['HP']/10           #(Change in ms(%) for change in HP in 10)
df2['Range'] = df2['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [283]:
# Creating dummy for china
df2['China'] = (df2['Country'] == 'CN').astype(int)

In [284]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# BLP instrument (sum)

In [285]:
fun.BLP(df2, 'HP')
fun.BLP(df2, 'Chargetime')
fun.BLP(df2, 'Range')

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,China,log_market_share,HP_sum,Chargetime_sum,Range_sum
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CN,257,1.0,1,-4.568514,1358.3,1861,2151.3
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CN,183,1.0,1,-5.119955,2033.9,2664,3218.0
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CN,177,1.0,1,-5.856883,2640.3,3241,4177.4
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CN,3,1.0,1,-9.934421,2639.0,3241,4176.9
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0,0,-3.203259,390.1,570,580.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,B,SE,4,1.0,0,-9.646739,2633.6,3247,4169.9
1163,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,C,SE,363,1.0,0,-4.223188,1338.2,1867,2145.6
1164,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,C,SE,1020,1.0,0,-3.401883,2013.8,2670,3212.3
1165,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,C,SE,1965,1.0,0,-3.449786,2620.2,3247,4171.7


In [286]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'HP', 'Chargetime']]
dummies = df2[[ 'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['Range_sum', 'HP_sum', 'Chargetime_sum']] 

In [287]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')

In [288]:
model.first_stage

,Price
R-squared,0.5822
Partial R-squared,0.0261
Shea's R-squared,0.0261
Partial F-statistic,17.145
P-value (Partial F-stat),0.0007
Partial F-stat Distn,chi2(3)
==========================,===========
const,-1.3281
,(-0.1735)
Range,0.1875


In [289]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -5.7877
Estimator:                    IV-2SLS   Adj. R-squared:                -5.8912
No. Observations:                 334   F-statistic:                    28.868
Date:                Mon, Apr 29 2024   P-value (F-stat)                0.0000
Time:                        16:59:45   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -2.8041     2.3134    -1.2121     0.2255     -7.3383      1.7300
Range          0.1995     0.0588     3.3923     0.0007      0.0842      0.3148
HP             0.3270     0.0955     3.4228     0.0006      0.1398      0.5143
Chargetime    -0.0848     0.0305    -2.7760     0.0055     -0.1447     -0.0249
China         -5.4735     1.1202    -4.8862     0.0000     -7.6690     -3.2779
Price         -0.3222     0.0761    -4.2313     0.0000     -0.4714     -0.1729
==============================================================================

Endogenous: Price
Instruments: Range_sum, HP_sum, Chargetime_sum
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde instrument (Local Diff)

In [290]:
fun.create_instrument_localsum(df2, 'HP', 0.5)
fun.create_instrument_localsum(df2, 'Chargetime', 0.5)
fun.create_instrument_localsum(df2, 'Range', 0.5)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Sales,const,China,log_market_share,HP_sum,Chargetime_sum,Range_sum,HP_instrument_localsum,Chargetime_instrument_localsum,Range_instrument_localsum
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,...,257,1.0,1,-4.568514,1358.3,1861,2151.3,740.5,1317,1499.9
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,...,183,1.0,1,-5.119955,2033.9,2664,3218.0,1060.2,1796,2153.5
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,...,177,1.0,1,-5.856883,2640.3,3241,4177.4,1501.0,2099,2792.3
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,...,3,1.0,1,-9.934421,2639.0,3241,4176.9,1456.2,2099,2746.8
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,...,222,1.0,0,-3.203259,390.1,570,580.9,310.5,533,470.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,...,4,1.0,0,-9.646739,2633.6,3247,4169.9,1777.6,1813,2189.4
1163,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,...,363,1.0,0,-4.223188,1338.2,1867,2145.6,1026.5,1138,1169.9
1164,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,...,1020,1.0,0,-3.401883,2013.8,2670,3212.3,1621.7,1483,1702.8
1165,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,...,1965,1.0,0,-3.449786,2620.2,3247,4171.7,1978.6,1813,2206.7


In [291]:
z=df2[['Range_instrument_localsum', 'HP_instrument_localsum', 'Chargetime_instrument_localsum']] 
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
model.first_stage

,Price
R-squared,0.6105
Partial R-squared,0.0919
Shea's R-squared,0.0919
Partial F-statistic,22.829
P-value (Partial F-stat),4.383e-05
Partial F-stat Distn,chi2(3)
================================,===========
const,1.7422
,(0.2392)
Range,0.4165


In [292]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -1.1894
Estimator:                    IV-2SLS   Adj. R-squared:                -1.2228
No. Observations:                 334   F-statistic:                    73.809
Date:                Mon, Apr 29 2024   P-value (F-stat)                0.0000
Time:                        16:59:46   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -4.6767     1.4901    -3.1386     0.0017     -7.5972     -1.7562
Range          0.1395     0.0231     6.0487     0.0000      0.0943      0.1847
HP             0.1587     0.0585     2.7127     0.0067      0.0440      0.2734
Chargetime    -0.0478     0.0201    -2.3749     0.0176     -0.0872     -0.0083
China         -3.7943     0.6345    -5.9798     0.0000     -5.0380     -2.5507
Price         -0.1722     0.0448    -3.8463     0.0001     -0.2599     -0.0844
==============================================================================

Endogenous: Price
Instruments: Range_instrument_localsum, HP_instrument_localsum, Chargetime_instrument_localsum
Robust Covariance (Heteroskedastic)
Debiased: False
"""